In [1]:
import warnings
warnings.filterwarnings('ignore')

# Libraries

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.stats import zscore
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GroupShuffleSplit
from sklearn.base import clone
from sklearn.linear_model import Lasso

from stabl.stabl import Stabl, plot_stabl_path, plot_fdr_graph, save_stabl_results, export_stabl_to_csv
from stabl.preprocessing import LowInfoFilter, remove_low_info_samples

%config InlineBackend.figure_formats=['retina']

In [ ]:
from stabl.multi_omic_pipelines import multi_omic_stabl, multi_omic_stabl_cv, late_fusion_lasso_cv
from stabl.single_omic_pipelines import single_omic_stabl, single_omic_stabl_cv
from stabl.pipelines_utils import compute_features_table

# Data

In [3]:
Celldensities =     pd.read_csv('../DataBatchcorrected/Harmony_Celldensities.csv', index_col=0)
Function =          pd.read_csv('../DataBatchcorrected/Harmony_Function.csv', index_col=0)
Metavariables =     pd.read_csv('../DataBatchcorrected/Harmony_Metavariables.csv', index_col=0)
Neighborhood =      pd.read_csv('../DataBatchcorrected/Harmony_Neighborhood.csv', index_col=0)

data = {
    'Celldensities': Celldensities,
    'Function': Function,
    'Metavariables': Metavariables,
    'Neighborhood': Neighborhood
}

y = pd.read_csv('../DataBatchcorrected/Outcome.csv',index_col=0)

In [4]:
y['patient_id'] = y.index.str.split('_').str.get(0)
y['site'] = y.index.to_series().apply(lambda x: 'Stanford' if x.startswith('S') else 'UOP')
y

,grade,patient_id,site
OC01_001,1,OC01,UOP
OC01_002,1,OC01,UOP
OC01_003,1,OC01,UOP
OC02_001,2,OC02,UOP
OC02_002,2,OC02,UOP
...,...,...,...
S23_2,1,S23,Stanford
S23_3,1,S23,Stanford
S24_1,1,S24,Stanford
S24_2,1,S24,Stanford


In [5]:
unique_patients = pd.DataFrame(y['patient_id'].unique(), columns=['patient_id'])
unique_patients = unique_patients.merge(y, on='patient_id', how = 'left') 
unique_patients = unique_patients.drop_duplicates(subset=['patient_id', 'grade', 'site'])
unique_patients

,patient_id,grade,site
0,OC01,1,UOP
3,OC02,2,UOP
5,OC03,2,UOP
8,OC04,2,UOP
11,OC05,2,UOP
14,OC06,2,UOP
17,OC07,2,UOP
20,OC08,2,UOP
23,OC09,1,UOP
26,OC10,1,UOP


In [6]:
# Split the dataframe into training and validation sets
train_df, val_df = train_test_split(unique_patients, test_size=0.25, stratify=unique_patients[['site', 'grade']], random_state=1)
train_df
# Print the shapes of the training and validation sets
print("Training set shape:", train_df.shape)
print("Validation set shape:", val_df.shape)
train_indices = y[y['patient_id'].isin(train_df['patient_id'])].index
train_indices

Training set shape: (36, 3)
Validation set shape: (12, 3)


Index(['OC01_001', 'OC01_002', 'OC01_003', 'OC04_001', 'OC04_002', 'OC04_003',
       'OC05_001', 'OC05_002', 'OC05_003', 'OC06_001',
       ...
       'S19_2', 'S19_3', 'S20_1', 'S20_2', 'S23_1', 'S23_2', 'S23_3', 'S24_1',
       'S24_2', 'S24_3'],
      dtype='object', length=107)

In [10]:
groups = y[y['patient_id'].isin(train_df['patient_id'])]
outer_groups = groups.patient_id.to_numpy().flatten()

array(['OC01', 'OC01', 'OC01', 'OC04', 'OC04', 'OC04', 'OC05', 'OC05',
       'OC05', 'OC06', 'OC06', 'OC06', 'OC07', 'OC07', 'OC07', 'OC08',
       'OC08', 'OC08', 'OC09', 'OC09', 'OC09', 'OC11', 'OC11', 'OC11',
       'OC12', 'OC12', 'OC12', 'OC17', 'OC17', 'OC17', 'OC18', 'OC18',
       'OC18', 'OC19', 'OC19', 'OC19', 'OC20', 'OC20', 'OC20', 'OC21',
       'OC21', 'OC21', 'OC24', 'OC24', 'OC24', 'OC25', 'OC25', 'OC25',
       'OC26', 'OC26', 'OC26', 'OC27', 'OC27', 'OC27', 'S01', 'S01',
       'S01', 'S02', 'S02', 'S02', 'S03', 'S03', 'S03', 'S04', 'S04',
       'S04', 'S06', 'S06', 'S06', 'S07', 'S07', 'S07', 'S08', 'S08',
       'S08', 'S09', 'S09', 'S09', 'S11', 'S11', 'S11', 'S12', 'S12',
       'S12', 'S13', 'S13', 'S13', 'S15', 'S15', 'S15', 'S16', 'S16',
       'S16', 'S18', 'S18', 'S18', 'S19', 'S19', 'S19', 'S20', 'S20',
       'S23', 'S23', 'S23', 'S24', 'S24', 'S24'], dtype=object)

In [16]:
# Split each dataframe in the data dictionary into train and test
train_data_dict = {}
test_data_dict = {}

for key, df in data.items():
    train_df = df.loc[train_indices]  # Select rows from the dataframe based on train indices
    test_df = df.drop(train_indices)  # Drop rows from the dataframe based on train indices
    
    train_data_dict[key] = train_df
    test_data_dict[key] = test_df

train_outcome = y.loc[train_indices].grade-1
test_outcome = y.drop(train_indices).grade-1

In [20]:
#Zscore training data
for data_name, data_frame in train_data_dict.items():
    numeric_columns = data_frame.select_dtypes(include=['float64', 'int64']).columns
    train_data_dict[data_name][numeric_columns] = train_data_dict[data_name][numeric_columns].apply(zscore)

# Calculate mean and SD from training data
train_means = {}
train_stds = {}
for data_name, data_frame in train_data_dict.items():
    numeric_columns = data_frame.select_dtypes(include=['float64', 'int64']).columns
    train_means[data_name] = data_frame[numeric_columns].mean()
    train_stds[data_name] = data_frame[numeric_columns].std()

# Normalize each data layer in testing data using the mean and standard deviation from training data
for data_name, data_frame in test_data_dict.items():
    if data_name in train_data_dict:
        numeric_columns = data_frame.select_dtypes(include=['float64', 'int64']).columns
        data_frame[numeric_columns] = (data_frame[numeric_columns] - train_means[data_name]) / train_stds[data_name]

# Results folder

In [ ]:
result_folder = "./RS_MC_RP"

# Main script

In [ ]:
for omic_name, X_omic in train_data_dict.items():
    X_omic = remove_low_info_samples(X_omic)
    train_data_dict[omic_name] = X_omic

In [ ]:
stabl = Stabl(
    lambda_name='C',
    lambda_grid=np.linspace(0.01, 5, 10),
    n_bootstraps=500,
    artificial_type="random_permutation",
    artificial_proportion=1.,
    replace=False,
    fdr_threshold_range=np.arange(0.2, 1, 0.01),
    sample_fraction=.5,
    random_state=111
 )

outer_splitter = GroupShuffleSplit(n_splits=100, test_size=.2, random_state=111)

stability_selection = clone(stabl).set_params(artificial_type=None, hard_threshold=0.5)

# Multi-omic Training-CV

In [ ]:
np.random.seed(111)

In [ ]:
predictions_dict = multi_omic_stabl_cv(
    data_dict=train_data_dict,
    y=train_outcome,
    outer_splitter=outer_splitter,
    stabl=stabl,
    stability_selection=stability_selection,
    task_type="binary",
    save_path=Path(result_folder),
    outer_groups=outer_groups
)

# Multiomic Training to derive coefficients

In [ ]:
np.random.seed(111)

In [ ]:
stabl_multi = Stabl(
    lambda_grid=np.linspace(0.01, 5, 30),
    n_bootstraps=5000,
    artificial_proportion=1.,
    artificial_type="random_permutation",
    hard_threshold=None,
    replace=False,
    fdr_threshold_range=np.arange(0.2, 1, 0.01),
    sample_fraction=.5,
    random_state=111
)

stability_selection = clone(stabl_multi).set_params(artificial_type=None, hard_threshold=.3)

In [ ]:
predictions_dict = multi_omic_stabl(
    data_dict=train_data_dict,
    y=train_outcome,
    stabl=stabl_multi,
    stability_selection=stability_selection,
    task_type="binary",
    save_path=Path(result_folder),
    X_test=pd.concat(test_data_dict.values(),axis=1),
    y_test=test_outcome
)

# Features Table

In [ ]:
selected_features_dict = dict()
for model in ["STABL", "EF Lasso", "SS 03", "SS 05", "SS 08"]:
    path = Path(result_folder, "Training-Validation", f"{model} coefficients.csv")
    try:
        selected_features_dict[model] = list(pd.read_csv(path, index_col=0).iloc[:, 0].index)
    except:
        selected_features_dict[model] = []

In [ ]:
features_table = compute_features_table(
    selected_features_dict,
    X_train=pd.concat(train_data_dict.values(), axis=1),
    y_train=train_outcome,
    task_type="binary"
)

In [ ]:
features_table.to_csv(Path(result_folder, "Training-Validation", "Table of features.csv"))